In [1]:
# Cell 1: Library Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import roc_auc_score
import logging
import os
from datetime import datetime
import klib
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
import xgboost as xgb


In [2]:
# Cell 2: Setup Logging
class Logger:
    def __init__(self):
        self.logger = self.setup_logging()
    
    def setup_logging(self):
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        log_file_name = f'training_{timestamp}.log'

        if os.path.exists('training.log'):
            os.remove('training.log')

        logger = logging.getLogger(__name__)
        logger.setLevel(logging.INFO)

        console_handler = logging.StreamHandler()
        file_handler = logging.FileHandler(log_file_name)

        formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        console_handler.setFormatter(formatter)
        file_handler.setFormatter(formatter)

        logger.addHandler(console_handler)
        logger.addHandler(file_handler)
        return logger
    
    def info(self, message):
        self.logger.info(message)
    
    def error(self, message):
        self.logger.error(message)

logger = Logger()
logger.info("Setup and Imports complete.")


2024-07-14 00:46:13,556 - __main__ - INFO - Setup and Imports complete.


In [3]:
# Cell 3: Data Handling Class
class DataHandler:
    def __init__(self, logger):
        self.logger = logger

    def load_data(self, train_path, test_path):
        try:
            train_df = pd.read_csv(train_path)
            test_df = pd.read_csv(test_path)
            self.logger.info("Datasets loaded successfully.")
            return train_df, test_df
        except Exception as e:
            self.logger.error(f"Error loading datasets: {e}")
            return None, None

    def rename_columns(self, df, column_mapping):
        df.rename(columns=column_mapping, inplace=True)
        self.logger.info("Columns renamed.")
        return df

data_handler = DataHandler(logger)
train_df, test_df = data_handler.load_data("klib_full_trainset.csv", "klib_full_testset.csv")

# Ensure datasets are loaded correctly
if train_df is not None and test_df is not None:
    new_column_names = {
        'gender': 'Gender',
        'age': 'Age',
        'driving_license': 'Driving_License',
        'region_code': 'Region_Code',
        'previously_insured': 'Previously_Insured',
        'vehicle_age': 'Vehicle_Age',
        'vehicle_damage': 'Vehicle_Damage',
        'annual_premium': 'Annual_Premium',
        'policy_sales_channel': 'Policy_Sales_Channel',
        'vintage': 'Vintage',
        'response': 'Response'
    }

    train_df = data_handler.rename_columns(train_df, new_column_names)
    test_df = data_handler.rename_columns(test_df, new_column_names)

    # Split the data into training and validation sets before preprocessing
    X = train_df.drop(columns=['Response'])
    y = train_df['Response']
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    logger.info(f"Training set shape: {X_train.shape}")
    logger.info(f"Validation set shape: {X_val.shape}")
else:
    raise ValueError("Failed to load datasets. Check the file paths and try again.")


2024-07-14 00:46:25,148 - __main__ - INFO - Datasets loaded successfully.
2024-07-14 00:46:25,151 - __main__ - INFO - Columns renamed.
2024-07-14 00:46:25,152 - __main__ - INFO - Columns renamed.
2024-07-14 00:46:30,173 - __main__ - INFO - Training set shape: (9172186, 9)
2024-07-14 00:46:30,174 - __main__ - INFO - Validation set shape: (2293047, 9)


In [4]:
# Cell 4: Feature Engineering Classes
class InteractionFeatures(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.feature_names = None

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X['Age_Annual_Premium'] = X['Age'] * X['Annual_Premium']
        X['Age_Vintage'] = X['Age'] * X['Vintage']
        X['Annual_Premium_Vintage'] = X['Annual_Premium'] * X['Vintage']
        X['Age_Region_Code'] = X['Age'] * X['Region_Code']
        X['Vintage_Region_Code'] = X['Vintage'] * X['Region_Code']
        X['Annual_Premium_Region_Code'] = X['Annual_Premium'] * X['Region_Code']
        self.feature_names = X.columns.tolist()
        return X

class PolynomialFeatureGeneration(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
        self.feature_names = None

    def fit(self, X, y=None):
        self.poly.fit(X[['Age', 'Annual_Premium', 'Vintage']])
        return self

    def transform(self, X):
        poly_features = self.poly.transform(X[['Age', 'Annual_Premium', 'Vintage']])
        poly_feature_names = self.poly.get_feature_names_out(['Age', 'Annual_Premium', 'Vintage'])
        poly_df = pd.DataFrame(poly_features, columns=[f'poly_{name.replace(" ", "_")}' for name in poly_feature_names], index=X.index)
        X = pd.concat([X, poly_df], axis=1)
        self.feature_names = X.columns.tolist()
        return X

logger.info("Custom transformers defined.")


2024-07-14 00:46:30,184 - __main__ - INFO - Custom transformers defined.


In [5]:
# Cell 5: Preprocessing and Model Preparation
class PreprocessingPipeline:
    def __init__(self, logger):
        self.logger = logger
        self.pipeline = self.create_pipeline()

    def create_pipeline(self):
        pipeline = Pipeline([
            ('interactions', InteractionFeatures()),
            ('poly_features', PolynomialFeatureGeneration()),
            ('scaling', StandardScaler(with_mean=False))  # Preserve feature names
        ])
        self.logger.info("Preprocessing pipeline defined.")
        return pipeline

    def preprocess_data(self, X_train, X_val, y_train):
        X_train_preprocessed = self.pipeline.fit_transform(X_train, y_train)
        X_val_preprocessed = self.pipeline.transform(X_val)

        feature_names = self.pipeline.named_steps['poly_features'].feature_names
        X_train_preprocessed = pd.DataFrame(X_train_preprocessed, columns=feature_names)
        X_val_preprocessed = pd.DataFrame(X_val_preprocessed, columns=feature_names)

        self.logger.info(f"Training set after preprocessing: {X_train_preprocessed.shape}")
        self.logger.info(f"Validation set after preprocessing: {X_val_preprocessed.shape}")
        return X_train_preprocessed, X_val_preprocessed

# Usage Example
preprocessor = PreprocessingPipeline(logger)
X_train_preprocessed, X_val_preprocessed = preprocessor.preprocess_data(X_train, X_val, y_train)

2024-07-14 00:46:30,197 - __main__ - INFO - Preprocessing pipeline defined.
2024-07-14 00:46:35,547 - __main__ - INFO - Training set after preprocessing: (9172186, 21)
2024-07-14 00:46:35,548 - __main__ - INFO - Validation set after preprocessing: (2293047, 21)


In [6]:
# Cell 6: Hyperparameter Optimization with Optuna
def objective(trial):
    param = {
        'eval_metric': 'auc',
        'early_stopping_rounds': 100,
        'lambda': trial.suggest_float('lambda', 1e-3, 10.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-3, 10.0, log=True),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4, 0.5, 0.6, 0.7, 0.8, 1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008, 0.01, 0.012, 0.014, 0.016, 0.018, 0.02]),
        'n_estimators': trial.suggest_int('n_estimators', 5, 1000),
        'max_depth': trial.suggest_categorical('max_depth', [5, 7, 9, 11, 13, 15, 17]),
        'random_state': 42,
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }
    
    y_train_series = pd.Series(y_train)  # Convert y_train to pandas Series
    ratio = float(y_train_series.value_counts()[0]) / y_train_series.value_counts()[1]
    model = xgb.XGBClassifier(**param, scale_pos_weight=ratio)
    
    model.fit(
        X_train_preprocessed, y_train_series,
        eval_set=[(X_val_preprocessed, y_val)],
        verbose=True
    )
    
    y_pred = model.predict_proba(X_val_preprocessed)[:, 1]
    auc = roc_auc_score(y_val, y_pred)
    return auc

logger.info("Starting hyperparameter optimization with Optuna.")
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

logger.info(f"Best trial parameters: {study.best_trial.params}")
logger.info(f"Best trial AUC: {study.best_trial.value}")


2024-07-14 00:46:35,558 - __main__ - INFO - Starting hyperparameter optimization with Optuna.
[I 2024-07-14 00:46:35,559] A new study created in memory with name: no-name-b07b39f3-c720-4491-9cc0-daca6ea42c42


[0]	validation_0-auc:0.86208
[1]	validation_0-auc:0.86274
[2]	validation_0-auc:0.86380
[3]	validation_0-auc:0.86382
[4]	validation_0-auc:0.86373
[5]	validation_0-auc:0.86373
[6]	validation_0-auc:0.86373
[7]	validation_0-auc:0.86398
[8]	validation_0-auc:0.86395
[9]	validation_0-auc:0.86398
[10]	validation_0-auc:0.86395
[11]	validation_0-auc:0.86395
[12]	validation_0-auc:0.86411
[13]	validation_0-auc:0.86408
[14]	validation_0-auc:0.86413
[15]	validation_0-auc:0.86416
[16]	validation_0-auc:0.86416
[17]	validation_0-auc:0.86415
[18]	validation_0-auc:0.86416
[19]	validation_0-auc:0.86416
[20]	validation_0-auc:0.86419
[21]	validation_0-auc:0.86429
[22]	validation_0-auc:0.86429
[23]	validation_0-auc:0.86429
[24]	validation_0-auc:0.86437
[25]	validation_0-auc:0.86443
[26]	validation_0-auc:0.86448
[27]	validation_0-auc:0.86453
[28]	validation_0-auc:0.86453
[29]	validation_0-auc:0.86453
[30]	validation_0-auc:0.86456
[31]	validation_0-auc:0.86458
[32]	validation_0-auc:0.86458
[33]	validation_0-au

[I 2024-07-14 00:54:19,932] Trial 0 finished with value: 0.8769453978423168 and parameters: {'lambda': 0.004302549695256806, 'alpha': 3.510644059149492, 'colsample_bytree': 1.0, 'subsample': 0.7, 'learning_rate': 0.008, 'n_estimators': 580, 'max_depth': 13, 'min_child_weight': 186}. Best is trial 0 with value: 0.8769453978423168.


[0]	validation_0-auc:0.84021
[1]	validation_0-auc:0.86081
[2]	validation_0-auc:0.85968
[3]	validation_0-auc:0.85377
[4]	validation_0-auc:0.85665
[5]	validation_0-auc:0.85934
[6]	validation_0-auc:0.86109
[7]	validation_0-auc:0.85943
[8]	validation_0-auc:0.86036
[9]	validation_0-auc:0.85938
[10]	validation_0-auc:0.85710
[11]	validation_0-auc:0.85786
[12]	validation_0-auc:0.86008
[13]	validation_0-auc:0.86040
[14]	validation_0-auc:0.85992
[15]	validation_0-auc:0.85858
[16]	validation_0-auc:0.85983
[17]	validation_0-auc:0.86051
[18]	validation_0-auc:0.86063
[19]	validation_0-auc:0.86041
[20]	validation_0-auc:0.86080
[21]	validation_0-auc:0.86003
[22]	validation_0-auc:0.85904
[23]	validation_0-auc:0.85948
[24]	validation_0-auc:0.85981
[25]	validation_0-auc:0.86006
[26]	validation_0-auc:0.86025
[27]	validation_0-auc:0.85956
[28]	validation_0-auc:0.85977
[29]	validation_0-auc:0.86031
[30]	validation_0-auc:0.86082
[31]	validation_0-auc:0.86122
[32]	validation_0-auc:0.86160
[33]	validation_0-au

[I 2024-07-14 01:08:10,083] Trial 1 finished with value: 0.8780962564969836 and parameters: {'lambda': 0.33181469384367357, 'alpha': 0.39546097010226333, 'colsample_bytree': 0.4, 'subsample': 1.0, 'learning_rate': 0.008, 'n_estimators': 988, 'max_depth': 11, 'min_child_weight': 171}. Best is trial 1 with value: 0.8780962564969836.


...

[I 2024-07-14 03:54:59,696] Trial 17 finished with value: 0.8809029542665469 and parameters: {'lambda': 0.006472707460224358, 'alpha': 0.11045192142038793, 'colsample_bytree': 0.7, 'subsample': 0.7, 'learning_rate': 0.016, 'n_estimators': 912, 'max_depth': 13, 'min_child_weight': 237}. Best is trial 15 with value: 0.8810665547964497.


[0]	validation_0-auc:0.85861
[1]	validation_0-auc:0.86563
[2]	validation_0-auc:0.86664
[3]	validation_0-auc:0.86602
[4]	validation_0-auc:0.86716
[5]	validation_0-auc:0.86745
[6]	validation_0-auc:0.86802
[7]	validation_0-auc:0.86779
[8]	validation_0-auc:0.86812
[9]	validation_0-auc:0.86814
[10]	validation_0-auc:0.86848
[11]	validation_0-auc:0.86862
[12]	validation_0-auc:0.86887
[13]	validation_0-auc:0.86903
[14]	validation_0-auc:0.86914
[15]	validation_0-auc:0.86914
[16]	validation_0-auc:0.86914
[17]	validation_0-auc:0.86921
[18]	validation_0-auc:0.86927
[19]	validation_0-auc:0.86930
[20]	validation_0-auc:0.86932
[21]	validation_0-auc:0.86944
[22]	validation_0-auc:0.86945
[23]	validation_0-auc:0.86948
[24]	validation_0-auc:0.86949
[25]	validation_0-auc:0.86947
[26]	validation_0-auc:0.86952
[27]	validation_0-auc:0.86941
[28]	validation_0-auc:0.86943
[29]	validation_0-auc:0.86950
[30]	validation_0-auc:0.86958
[31]	validation_0-auc:0.86958
[32]	validation_0-auc:0.86961
[33]	validation_0-au

[I 2024-07-14 04:06:17,752] Trial 18 finished with value: 0.8805419347398724 and parameters: {'lambda': 0.003237815436660901, 'alpha': 9.333294705255685, 'colsample_bytree': 0.7, 'subsample': 0.7, 'learning_rate': 0.016, 'n_estimators': 647, 'max_depth': 15, 'min_child_weight': 209}. Best is trial 15 with value: 0.8810665547964497.


[0]	validation_0-auc:0.84568
[1]	validation_0-auc:0.85567
[2]	validation_0-auc:0.85615
[3]	validation_0-auc:0.85654
[4]	validation_0-auc:0.85733
[5]	validation_0-auc:0.85730
[6]	validation_0-auc:0.85837
[7]	validation_0-auc:0.85856
[8]	validation_0-auc:0.85891
[9]	validation_0-auc:0.85872
[10]	validation_0-auc:0.85861
[11]	validation_0-auc:0.85848
[12]	validation_0-auc:0.85888
[13]	validation_0-auc:0.85894
[14]	validation_0-auc:0.85909
[15]	validation_0-auc:0.85899
[16]	validation_0-auc:0.85888
[17]	validation_0-auc:0.85896
[18]	validation_0-auc:0.85900
[19]	validation_0-auc:0.85911
[20]	validation_0-auc:0.85923
[21]	validation_0-auc:0.85934
[22]	validation_0-auc:0.85942
[23]	validation_0-auc:0.85943
[24]	validation_0-auc:0.85941
[25]	validation_0-auc:0.85948
[26]	validation_0-auc:0.85952
[27]	validation_0-auc:0.85945
[28]	validation_0-auc:0.85948
[29]	validation_0-auc:0.85949
[30]	validation_0-auc:0.85954
[31]	validation_0-auc:0.85960
[32]	validation_0-auc:0.85957
[33]	validation_0-au

[I 2024-07-14 04:18:09,405] Trial 19 finished with value: 0.8778279849296188 and parameters: {'lambda': 0.014559541547030692, 'alpha': 0.11539996258701654, 'colsample_bytree': 0.7, 'subsample': 0.7, 'learning_rate': 0.018, 'n_estimators': 890, 'max_depth': 7, 'min_child_weight': 292}. Best is trial 15 with value: 0.8810665547964497.
2024-07-14 04:18:09,407 - __main__ - INFO - Best trial parameters: {'lambda': 0.0011124463405821116, 'alpha': 0.4697409790780375, 'colsample_bytree': 0.7, 'subsample': 0.7, 'learning_rate': 0.016, 'n_estimators': 923, 'max_depth': 17, 'min_child_weight': 221}
2024-07-14 04:18:09,408 - __main__ - INFO - Best trial AUC: 0.8810665547964497


In [7]:
# Train the final model using the best parameters
best_params = study.best_trial.params
final_model = xgb.XGBClassifier(**best_params)
final_model.fit(X_train_preprocessed, y_train)

# Validate the final model
final_preds = final_model.predict_proba(X_val_preprocessed)[:, 1]
final_auc = roc_auc_score(y_val, final_preds)
logger.info(f"Final model AUC on validation set: {final_auc}")

2024-07-14 04:26:29,272 - __main__ - INFO - Final model AUC on validation set: 0.8809612985184044


In [8]:
# Cell 7: Train the Final Model using DMatrix with Best Parameters
best_params = study.best_trial.params
dtrain = xgb.DMatrix(data=X_train_preprocessed, label=y_train)
dval = xgb.DMatrix(data=X_val_preprocessed, label=y_val)

final_model = xgb.train(
    params=best_params,
    dtrain=dtrain,
    num_boost_round=best_params['n_estimators'],
    evals=[(dval, 'eval')],
    early_stopping_rounds=100
)

# Validate the final model
final_preds = final_model.predict(dval)
final_auc = roc_auc_score(y_val, final_preds)
logger.info(f"Final model AUC on validation set: {final_auc}")

c:\Users\paulo\anaconda3\envs\pytorch_env\Lib\site-packages\xgboost\core.py:158: UserWarning: [04:26:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	eval-rmse:0.32748
[1]	eval-rmse:0.32642
[2]	eval-rmse:0.32542
[3]	eval-rmse:0.32487
[4]	eval-rmse:0.32398
[5]	eval-rmse:0.32304
[6]	eval-rmse:0.32212
[7]	eval-rmse:0.32159
[8]	eval-rmse:0.32068
[9]	eval-rmse:0.31987
[10]	eval-rmse:0.31903
[11]	eval-rmse:0.31829
[12]	eval-rmse:0.31751
[13]	eval-rmse:0.31674
[14]	eval-rmse:0.31600
[15]	eval-rmse:0.31533
[16]	eval-rmse:0.31463
[17]	eval-rmse:0.31394
[18]	eval-rmse:0.31329
[19]	eval-rmse:0.31265
[20]	eval-rmse:0.31202
[21]	eval-rmse:0.31146
[22]	eval-rmse:0.31086
[23]	eval-rmse:0.31032
[24]	eval-rmse:0.30977
[25]	eval-rmse:0.30922
[26]	eval-rmse:0.30871
[27]	eval-rmse:0.30845
[28]	eval-rmse:0.30794
[29]	eval-rmse:0.30741
[30]	eval-rmse:0.30689
[31]	eval-rmse:0.30643
[32]	eval-rmse:0.30595
[33]	eval-rmse:0.30553
[34]	eval-rmse:0.30507
[35]	eval-rmse:0.30465
[36]	eval-rmse:0.30425
[37]	eval-rmse:0.30406
[38]	eval-rmse:0.30362
[39]	eval-rmse:0.30323
[40]	eval-rmse:0.30287
[41]	eval-rmse:0.30249
[42]	eval-rmse:0.30215
[43]	eval-rmse:0.3018

2024-07-14 04:39:50,818 - __main__ - INFO - Final model AUC on validation set: 0.8797911836736267


In [2]:
optuna.visualization.plot_optimization_history(study)

NameError: name 'optuna' is not defined

In [10]:
optuna.visualization.plot_parallel_coordinate(study)

In [11]:
optuna.visualization.plot_slice(study)

In [12]:
optuna.visualization.plot_contour(study)

In [13]:
optuna.visualization.plot_param_importances(study)

In [1]:
study.best_trial

NameError: name 'study' is not defined